In [2]:
# import libraries
# %matplotlib notebook
# %matplotlib inline

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')
import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import demoji
import pickle
import re
import random
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
import tensorflow as tf
print(f'tensofrlow:{tf.__version__}')

import os
os.getcwd()

demoji.__version__
with open(r'../data/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}
    
from tqdm.notebook import tqdm as tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

import pickle

Library versions:
keras:2.8.0
pandas:1.3.4
sklearn:0.24.2
nltk:3.6.7
numpy:1.20.3
tensofrlow:2.8.0


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MeetParadia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# keras imports, because there are like... A million of them.
from keras.models import Model
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
     TimeDistributed
from keras.utils import np_utils

# from tensorflow.keras import Model, Sequential
# from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
#      TimeDistributed

In [4]:
#!pip install bert_score

In [5]:
import bert_score
bert_score.__version__

'0.3.11'

In [6]:
from bert_score import score
from bert_score import BERTScorer

In [7]:
model = keras.models.load_model('C:/Users/MeetParadia/Desktop/NLP/model/s2s_model.h5')

In [8]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
    
with open('C:/Users/MeetParadia/Desktop/NLP/model/testing_data.pickle', 'rb') as handle:
    testing_data = pickle.load(handle)

In [9]:
#!pip install rouge

In [10]:
from nltk.translate.meteor_score import meteor_score as meteor_score
from rouge import Rouge
import nltk.translate.bleu_score as bleu

# model parameters

In [368]:
# 8192 - large enough for demonstration, larger values make network training slower
MAX_VOCAB_SIZE = 2**13
# seq2seq generally relies on fixed length message vectors - longer messages provide more info
# but result in slower training and larger networks
MAX_MESSAGE_LEN = 30  
# Embedding size for words - gives a trade off between expressivity of words and network size
EMBEDDING_SIZE = 100
# Embedding size for whole messages, same trade off as word embeddings
CONTEXT_SIZE = 100
# Larger batch sizes generally reach the average response faster, but small batch sizes are
# required for the model to learn nuanced responses.  Also, GPU memory limits max batch size.
BATCH_SIZE = 4
# Helps regularize network and prevent overfitting.
DROPOUT = 0.2
# High learning rate helps model reach average response faster, but can make it hard to 
# converge on nuanced responses
LEARNING_RATE=0.005

# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted

# Implementaiton detail for allowing this to be run in Kaggle's notebook hardware
SUB_BATCH_SIZE = 1000

# data preparation 

In [369]:
count_vec = CountVectorizer(tokenizer=casual_tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit(tqdm(training_data.loc[:,'text_x'].to_list() + training_data.loc[:,'text_y'].to_list()))
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START

# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

Fitting CountVectorizer on X and Y text data...


  0%|          | 0/190632 [00:00<?, ?it/s]

Learned vocab of 8192 items.


In [370]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()

In [371]:
x_text = training_data.loc[:,'text_x']
y_text = training_data.loc[:,'text_y']
print("Calculating word indexes for X...")
train_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
train_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

Calculating word indexes for X...


  0%|          | 0/95316 [00:00<?, ?it/s]

Calculating word indexes for Y...


  0%|          | 0/95316 [00:00<?, ?it/s]

In [372]:
x_text = testing_data.loc[:,'text_x']
y_text = testing_data.loc[:,'text_y']
print("Calculating word indexes for X...")
test_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
test_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

Calculating word indexes for X...


  0%|          | 0/23829 [00:00<?, ?it/s]

Calculating word indexes for Y...


  0%|          | 0/23829 [00:00<?, ?it/s]

# Model Structure

In [373]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])
def train_mini_epoch(model, start_idx, end_idx):
    """ Batching seems necessary in Kaggle Jupyter Notebook environments, since
        `model.fit` seems to freeze on larger batches (somewhere 1k-10k).
    """
    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    print('Test results:', model.evaluate(
        [test_x[rand_idx], add_start_token(test_y[rand_idx])],
        binarize_labels(test_y[rand_idx])
    ))
    
    input_strings = [
        "@AppleSupport I fix I this I stupid I problem I",
        "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service.",
    ]
    
    for input_string in input_strings:
        output_string = respond_to(model, input_string)
        print(f'> "{input_string}"\n< "{output_string}"')

In [316]:
respond_to(model, "@AppleSupport testing testing!")
#respond_to(model, "Guess what airline I won’t be flying anytime soon?  @AlaskaAir #ShameOnYou __URL__")

"@__sn__ we want to help . dm us and we'll go from there . __url__"

In [374]:
def get_pred(text):
    return respond_to(model, text)

In [375]:
len(testing_data)

23829

In [76]:
pred_x = []
for text in range(0, len(testing_data)):
    pred_x.append(get_pred(testing_data['text_x'].iloc[text]))

In [78]:
pred_x

['@__sn__ thanks for the shout out ! - sara',
 '@__sn__ hi , __unk__ . we are not able to assist you with your reservation . please dm your confirmation number . * amv __url__',
 "@__sn__ hi __unk__ , i'm sorry to hear that . can you dm us your order number , full name , address and email address ? thanks - chloe",
 "@__sn__ hey there ! can you dm us your account's email address or username ? we'll take a look backstage / mt __url__",
 '@__sn__ hi __unk__ , sorry to hear this , __unk__ . if you still have the receipt and packaging , you can return the packaging to any of our team',
 "@__sn__ hi there ! we don't have any info on this right now , but we'll let the right team know it's something you'd like to see / df",
 "@__sn__ hi __unk__ , we're sorry to hear about this . please dm us your contact info and store address . thanks !",
 "@__sn__ hi __unk__ , i'm sorry to hear that you are not able to purchase your order . can you please dm your full name , address and email ?",
 '@__sn__ 

In [52]:
reference_y = testing_data['text_y'].tolist()

In [320]:
print(len(pred_x))
print(len(reference_y))

23829
23829


In [104]:
def get_random():
    random.seed(102)
    x = [random.randint(0, 23829) for i in range(0, 2383)]
    return x

In [105]:
index = get_random()

In [378]:
results_df = pd.DataFrame(
    {'pred_x': pred_x,
     'reference_y': reference_y
    })

In [379]:
mini_results = results_df.iloc[index]

In [380]:
mini_pred_x = mini_results['pred_x'].tolist()
mini_reference_y = mini_results['reference_y'].tolist()

In [381]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/pred_x.pickle', 'wb') as handle:
    pickle.dump(pred_x, handle)
with open('C:/Users/MeetParadia/Desktop/NLP/model/reference_y.pickle', 'wb') as handle:
    pickle.dump(reference_y, handle)

2383

In [49]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/mini_pred_x.pickle', 'wb') as handle:
    pickle.dump(mini_pred_x, handle)
with open('C:/Users/MeetParadia/Desktop/NLP/model/mini_reference_y.pickle', 'wb') as handle:
    pickle.dump(mini_reference_y, handle)

In [54]:
# with open('C:/Users/MeetParadia/Desktop/NLP/model/pred_x.pickle', 'rb') as handle:
#     pred_x = pickle.load(handle)
    
# with open('C:/Users/MeetParadia/Desktop/NLP/model/reference_y.pickle', 'rb') as handle:
#     reference_y = pickle.load(handle)

In [22]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/mini_pred_x.pickle', 'rb') as handle:
    mini_pred_x = pickle.load(handle)
    
with open('C:/Users/MeetParadia/Desktop/NLP/model/mini_reference_y.pickle', 'rb') as handle:
    mini_reference_y = pickle.load(handle)

In [12]:
def get_bert_score(cand, ref):
    scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    P, R, F1 = scorer.score([cand], [ref])
    return F1

# def get_meteor_score(cand, ref):
#     cand1 = [cand.split()]
#     ref1 = ref.split()
#     return round(meteor_score(cand1, ref1))

def get_rouge_score(cand, ref):
    rouge = Rouge()
    return rouge.get_scores(cand, ref)

def get_bleu_score(cand, ref):
    return bleu.sentence_bleu([cand], ref)

In [13]:
len(mini_pred_x)

2383

In [27]:
bert_score = []
for i in range(0, len(mini_pred_x)):
    x = mini_pred_x[i]
    y = mini_reference_y[i]
    bert_score.append(get_bert_score(x, y))
    print(i)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

2380


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2381


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2382


In [18]:
rouge_score = []
for i in range(0, len(mini_pred_x)):
    x = mini_pred_x[i]
    y = mini_reference_y[i]
    rouge_score.append(get_rouge_score(x, y))

In [289]:
pd.DataFrame(rouge_score).to_csv('C:/Users/MeetParadia/Desktop/NLP/model/rouge_score.csv')

In [24]:
bleu_score = []
for i in range(0, len(mini_pred_x)):
    x = mini_pred_x[i]
    y = mini_reference_y[i]
    bleu_score.append(get_bleu_score(x, y))

In [25]:
bleu_score

[0.18252713693173594,
 0.27840589430020246,
 0.23450567189365337,
 0.5954446334983143,
 0.2121338772930392,
 0.6512999014509429,
 0.20498849712517386,
 0.23814649672673863,
 0.2945051974867753,
 0.107280299253807,
 0.17110951507255034,
 0.24870980777683308,
 0.31171645409650817,
 0.08898914444666245,
 0.3836347187254158,
 0.13987585618506887,
 0.13218082824706753,
 0.33809757465471396,
 0.25784113475955434,
 0.7052191141067626,
 0.21132460314473342,
 0.32916765312633284,
 0.23765393864467171,
 0.36953963801782347,
 0.4623383665773071,
 0.1377904098896197,
 0.08824682743016699,
 0.22031482177541462,
 0.28798566219520044,
 0.14855384326797727,
 0.13798184114481785,
 0.07987763327992783,
 0.2703587851920814,
 0.32818675982670314,
 0.04558280353020834,
 0.1797722590442,
 0.38516622116155025,
 0.08405001772213613,
 0.8127029214600074,
 0.22876461155349287,
 0.2756562874882709,
 0.2803560117171275,
 0.13602289673980306,
 0.25503541144909664,
 0.06637330676100489,
 0.25099948696060914,
 0.239

In [26]:
np.array(bleu_score).mean()

0.2653732499932876

In [28]:
bert_score

[tensor([0.1166]),
 tensor([0.4419]),
 tensor([0.2226]),
 tensor([0.6735]),
 tensor([0.2910]),
 tensor([0.5816]),
 tensor([0.1259]),
 tensor([0.3730]),
 tensor([0.1952]),
 tensor([-0.2707]),
 tensor([0.2471]),
 tensor([0.2667]),
 tensor([0.2611]),
 tensor([0.1273]),
 tensor([0.1011]),
 tensor([0.1507]),
 tensor([0.1250]),
 tensor([0.2813]),
 tensor([0.1384]),
 tensor([0.6679]),
 tensor([0.1216]),
 tensor([0.3352]),
 tensor([0.1616]),
 tensor([0.4396]),
 tensor([0.4133]),
 tensor([0.0838]),
 tensor([0.0453]),
 tensor([0.2844]),
 tensor([0.1463]),
 tensor([-0.0828]),
 tensor([0.2277]),
 tensor([-0.0802]),
 tensor([0.4421]),
 tensor([0.2307]),
 tensor([0.0308]),
 tensor([0.0863]),
 tensor([0.2571]),
 tensor([-0.5103]),
 tensor([0.8014]),
 tensor([0.1189]),
 tensor([0.1351]),
 tensor([0.2060]),
 tensor([0.1160]),
 tensor([0.3382]),
 tensor([0.0970]),
 tensor([0.2763]),
 tensor([0.2378]),
 tensor([0.2846]),
 tensor([0.1175]),
 tensor([0.1831]),
 tensor([0.1066]),
 tensor([0.3241]),
 tensor(

In [29]:
import tensorflow as tf

In [30]:
bert_score_np0 = bert_score[0].numpy()

In [31]:
bert_score_np0

array([0.11663456], dtype=float32)

In [32]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/bert_score.pickle', 'wb') as handle:
    pickle.dump(bert_score, handle)

In [36]:
pd.DataFrame(bert_score).to_csv('C:/Users/MeetParadia/Desktop/NLP/model/bert_score.csv')

In [43]:
bert_score_list = []
for i in bert_score:
    bert_score_list.append(float(i.numpy()))

In [47]:
np.array(bert_score_list).mean()

0.20209711935809171

In [46]:
with open('C:/Users/MeetParadia/Desktop/NLP/model/bert_score_list.pickle', 'wb') as handle:
    pickle.dump(bert_score_list, handle)

In [67]:
rouge_score[0][0]#['rouge-1']['r']

{'rouge-1': {'r': 0.16, 'p': 0.3333333333333333, 'f': 0.21621621183345519},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.16, 'p': 0.3333333333333333, 'f': 0.21621621183345519}}

In [114]:
rouge_type = ['rouge-1', 'rouge-2', 'rouge-l']
metric_type = ['r', 'p', 'f']

rouge_1_recall = []
rouge_2_recall = []
rouge_l_recall = []

for ele in range(0, len(rouge_score)):
    for typ in rouge_type:
        for metric in metric_type:
            if metric == 'f':
                if typ == 'rouge-1':
                    rouge_1_recall.append(rouge_score[ele][0][typ][metric])
                elif typ == 'rouge-2':
                    rouge_2_recall.append(rouge_score[ele][0][typ][metric])
                else:
                    rouge_l_recall.append(rouge_score[ele][0][typ][metric])

# Results

## Rouge F1 score 

In [120]:
#Rouge 1:
print("Mean:", np.array(rouge_1_recall).mean())
print("Max:", np.array(rouge_1_recall).max())
print("Min:", np.array(rouge_1_recall).min())

max_ind = np.array(rouge_1_recall).argmax(axis=0)
min_ind = np.array(rouge_1_recall).argmin(axis=0)

print("\nBest match:")
print(mini_pred_x[max_ind])
print(mini_reference_y[max_ind])

print("Worst match:")
print(mini_pred_x[min_ind])
print(mini_reference_y[min_ind])

Mean: 0.2006186203624341
Max: 0.7741935433922998
Min: 0.03921568150711323

Best match:
@__sn__ here to help ! send us a note at __url__ so our team can connect .
 @__sn__ Here to help. Send us a note at __URL__ so our team can connect.
Worst match:
@__sn__ bonjour , je suis désolée d'apprendre cela . avez-vous __unk__ notre sav afin de __unk__ __unk__ sur notre site client ? ^ me
 @__sn__ Bonsoir Benjamin, nous  vous remercie pour ces informations. Nous félicitons vraiment votre reflexe et votre réactivité. Afin que nous puissions faire le nécessaire, veuillez nous transmettre tous ces éléments par email au __email__ ^LiB


In [121]:
#Rouge 2:
print("Mean:", np.array(rouge_2_recall).mean())
print("Max:", np.array(rouge_2_recall).max())
print("Min:", np.array(rouge_2_recall).min())

max_ind = np.array(rouge_2_recall).argmax(axis=0)
min_ind = np.array(rouge_2_recall).argmin(axis=0)

print("\nBest match:")
print(mini_pred_x[max_ind])
print(mini_reference_y[max_ind])

print("Worst match:")
print(mini_pred_x[min_ind])
print(mini_reference_y[min_ind])

Mean: 0.032996777128258596
Max: 0.6274509754402153
Min: 0.0

Best match:
@__sn__ i'm sorry for the delay . please dm us your tracking number , delivery address and contact number via the link so we can look into this for you
 @__sn__ Please DM us your tracking, delivery address and contact number via the link so we can look into this for you. ^CH __URL__
Worst match:
@__sn__ we want to help . dm us and we'll get started . __url__
 @__sn__ Hi Sergio! We're afraid we don't have any news on this right now, but we'll make sure to pass your thoughts on to the team /AU


In [123]:
#Rouge L:
print("Mean:", np.array(rouge_l_recall).mean())
print("Max:", np.array(rouge_l_recall).max())
print("Min:", np.array(rouge_l_recall).min())

max_ind = np.array(rouge_l_recall).argmax(axis=0)
min_ind = np.array(rouge_l_recall).argmin(axis=0)

print("\nBest match:")
print(mini_pred_x[max_ind])
print(mini_reference_y[max_ind])

print("Worst match:")
print(mini_pred_x[min_ind])
print(mini_reference_y[min_ind])

Mean: 0.1898533117760914
Max: 0.7741935433922998
Min: 0.03921568150711323

Best match:
@__sn__ here to help ! send us a note at __url__ so our team can connect .
 @__sn__ Here to help. Send us a note at __URL__ so our team can connect.
Worst match:
@__sn__ bonjour , je suis désolée d'apprendre cela . avez-vous __unk__ notre sav afin de __unk__ __unk__ sur notre site client ? ^ me
 @__sn__ Bonsoir Benjamin, nous  vous remercie pour ces informations. Nous félicitons vraiment votre reflexe et votre réactivité. Afin que nous puissions faire le nécessaire, veuillez nous transmettre tous ces éléments par email au __email__ ^LiB


## BLEU score 

In [81]:
print("Mean:", np.array(bleu_score).mean())
print("Max:", np.array(bleu_score).max())
print("Min:", np.array(bleu_score).min())

Mean: 0.2653732499932876
Max: 0.8417926997118741
Min: 8.96262662197079e-05


In [99]:
max_ind = np.array(bleu_score).argmax(axis=0)
min_ind = np.array(bleu_score).argmin(axis=0)

print("Best match:")
print(mini_pred_x[max_ind])
print(mini_reference_y[max_ind])

print("Worst match:")
print(mini_pred_x[min_ind])
print(mini_reference_y[min_ind])

Best match:
@__sn__ here ’ s what you can do to work around the issue until it ’ s fixed in a future software update : __url__
 @__sn__ Here’s what you can do to work around the issue until it’s fixed in a future software update: __URL__
Worst match:
@__sn__ hi , __unk__ . __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ . ^ vm
 @__sn__  __URL__


## BERT score 

In [82]:
bert_score_list = []
for i in bert_score:
    bert_score_list.append(float(i.numpy()))

In [83]:
print("Mean:", np.array(bert_score_list).mean())
print("Max:", np.array(bert_score_list).max())
print("Min:", np.array(bert_score_list).min())

Mean: 0.20209711935809171
Max: 0.8014407157897949
Min: -0.623305082321167


In [100]:
max_ind = np.array(bert_score_list).argmax(axis=0)
min_ind = np.array(bert_score_list).argmin(axis=0)

print("Best match:")
print(mini_pred_x[max_ind])
print(mini_reference_y[max_ind])

print("Worst match:")
print(mini_pred_x[min_ind])
print(mini_reference_y[min_ind])

Best match:
@__sn__ hi there . please follow us via twitter so we can send you a dm with further instructions . thanks !
 @__sn__ Hi there. Please follow us via Twitter so we can send you a DM with further instructions. Thanks!
Worst match:
@__sn__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__
 @__sn__ Hast du mal versucht das Gerät neu zu starten, oder die Inhalte zu aktualisieren? ^AN


In [124]:
metrics_df = pd.DataFrame(
    {'index': index,
     'mini_pred_x': mini_pred_x,
     'mini_reference_y': mini_reference_y,
     'bert_score': bert_score_list,
     'rouge_l_recall': rouge_l_recall,
     'bleu_score': bleu_score
    })

In [125]:
metrics_df.sort_values(by=['index'], inplace=True)

In [126]:
metrics_df

,index,mini_pred_x,mini_reference_y,bert_score,rouge_l_recall,bleu_score
2226,8,"@__sn__ hi , we are sorry to hear about this ....","@__sn__ HI Ravichandra, we have responded to ...",0.308280,0.350000,0.368796
1625,24,"@__sn__ hi , __unk__ . we don't have any info ...",@__sn__ Hi there weary_face What's happened...,0.015386,0.162162,0.245633
845,59,"@__sn__ hi there , we are aware of this issue ...",@__sn__ Hi I never heard back from you. Let m...,0.224275,0.051282,0.195169
333,79,@__sn__ hey there ! we don't have any info on ...,"@__sn__ Hey there, if you need assistance ple...",0.152366,0.081633,0.237532
2059,131,@__sn__ hi there ! we don't have any info on t...,@__sn__ Thank you for flagging this. Our team...,0.211587,0.208333,0.322423
...,...,...,...,...,...,...
290,23808,@__sn__ we want to help . dm us and we'll go f...,@__sn__ We’d be happy to help in any way we c...,0.220842,0.193548,0.257427
682,23810,"@__sn__ hi , we have replied to you via dm . ^...",@__sn__ then select Marketing messages and fo...,0.190341,0.071429,0.117847
1719,23814,@__sn__ we know that you love the best in the ...,@__sn__ We're happy that you'll have more tim...,0.205565,0.195122,0.291428
2096,23815,"@__sn__ hi , __unk__ . we don't have any info ...",@__sn__ We never mean to disappoint! Let's ch...,0.119308,0.117647,0.135072


In [ ]:
#pd.DataFrame(bert_score).to_csv('C:/Users/MeetParadia/Desktop/NLP/model/metrics_df.csv')

In [133]:
metrics_df.corr()

,index,bert_score,rouge_l_recall,bleu_score
index,1.000000,0.017767,-0.008011,-0.001818
bert_score,0.017767,1.000000,0.656003,0.679991
rouge_l_recall,-0.008011,0.656003,1.000000,0.780986
bleu_score,-0.001818,0.679991,0.780986,1.000000


In [134]:
pd.DataFrame(metrics_df.corr()).to_csv('C:/Users/MeetParadia/Desktop/NLP/model/metrics_df_corr.csv')

In [140]:
metrics_df[(metrics_df['bleu_score'] < 0.2) & (metrics_df['bert_score'] > 0.5)]

,index,mini_pred_x,mini_reference_y,bert_score,rouge_l_recall,bleu_score
558,19146,@__sn__ hi there ! we are sorry to hear about ...,"@__sn__ Hey, can you DM us your phone number?...",0.543827,0.285714,0.166945


In [145]:
metrics_df[(metrics_df['rouge_l_recall'] < 0.2) & (metrics_df['bert_score'] > 0.5)]

,index,mini_pred_x,mini_reference_y,bert_score,rouge_l_recall,bleu_score
747,534,@__sn__ that's concerning . please write us at...,@__sn__ That's troubling to hear. Please cont...,0.567129,0.086957,0.326780
2132,5728,@__sn__ that's concerning . please write us at...,@__sn__ Oh no! I'm so sorry this happened. Pl...,0.540000,0.160000,0.297969
259,11742,@__sn__ that's concerning . please write us at...,@__sn__ This is concerning. Please reach out ...,0.538869,0.173913,0.386014
